Привет, меня зовут Артем Хуршудов. Сегодня я проверю твой проект.
<br> Дальнейшее общение будет происходить на "ты" если это не вызывает никаких проблем.
<br> Желательно реагировать на каждый мой комментарий ('исправил', 'не понятно как исправить ошибку', ...)
<br> Пожалуйста, не удаляй комментарии ревьюера, так как они повышают качество повторного ревью.

Комментарии будут в <font color='green'>зеленой</font>, <font color='blue'>синей</font> или <font color='red'>красной</font> рамках:

<div class="alert alert-block alert-success">
<b>Успех:</b> Если все сделано отлично
</div>

<div class="alert alert-block alert-info">
<b>Совет: </b> Если можно немного улучшить
</div>

<div class="alert alert-block alert-danger">
<b>Ошибка:</b> Если требуются исправления. Работа не может быть принята с красными комментариями.
</div>

-------------------

Будет очень хорошо, если ты будешь помечать свои действия следующим образом:
<div class="alert alert-block alert-warning">
<b>Комментарий студента:</b> ...
</div>

<div class="alert alert-block alert-warning">
<b>Изменения:</b> Были внесены следующие изменения ...
</div>

### <font color='orange'>Общее впечатление</font>
* Большое спасибо за проделанную работу. Видно, что приложено много усилий.
* Выводы и рассуждения получились содержательными, их было интересно читать.
* Отлично, что риск убытков посчитан верно.
* Над этим проектом нужно будет еще немного поработать. Однако, изменения не должны занять много времени.
* С радостью отвечу на твои вопросы, если они есть. Лучше всего их собрать в следующей ячейке. Жду новую версию проекта.

<img src="http://s3.amazonaws.com/pix.iemoji.com/images/emoji/apple/ios-12/256/waving-hand.png" align=left width=44, heigth=44>
<div class="alert alert-block alert-warning">
<b> Обратная связь </b>
    
Спасибо большое за подробное ревью, Ваши замечания были очень полезны. 🙇
</div>

### <font color='orange'>Общее впечатление (ревью 2)</font>
* После исправлений проект улучшился и теперь он может быть зачтен.
* Теперь все статистики посчитаны корректно, молодец!
* Хорошо, что ты отметил в выводе наличие аномалий во втором регионе.
* Удачи в дальнейшем обучении и следующих работах!

# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Импрортируем необходимые библиотеки

In [ ]:
import sys
!{sys.executable} -m pip install missingno
#from IPython.lib.display import YouTubeVideo
import plotly.graph_objects as go
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime
import plotly.express as px
import random
import seaborn as sns
import missingno as msno
import operator
from scipy import stats as st
from sklearn.linear_model import  LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate, cross_val_predict
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.dummy import DummyRegressor
from sklearn.decomposition import PCA
from sklearn.utils import resample, shuffle
from sklearn.metrics import accuracy_score, mean_squared_error, f1_score, precision_score 
from sklearn.metrics import recall_score,roc_curve,precision_recall_curve

<div class="alert alert-block alert-success">
<b>Успех:</b> Отлично, что все импорты собраны в первой ячейке ноутбука! Если у того, кто будет запускать твой ноутбук будут отсутствовать некоторые библиотеки, то он это увидит сразу, а не в процессе!
</div>

## Методы и функции

In [ ]:
random_state = 12345
alpha = 0.01

### Метод check_destribution

Метод сравнивает три распределения по их гистограммам

In [ ]:
def check_destribution(series_0, series_1, series_2, names, title, logy=True):
    fig = go.Figure()
    fig.add_trace(go.Histogram(x=series_0, name=names[0], histnorm='density'))
    fig.add_trace(go.Histogram(x=series_1, name=names[1], histnorm='density'))
    fig.add_trace(go.Histogram(x=series_2, name=names[2], histnorm='density'))
    # Overlay both histograms
    fig.update_layout(barmode='overlay')
    # Reduce opacity to see all histograms
    fig.update_traces(opacity=0.75)
    if logy:
        fig.update_yaxes(type="log")
    fig.update_layout(title=title)
    fig.show()

### Метод pair_plot_data

Строит парные графики для целевого признака и исследуемых параметром

In [ ]:
def pair_plot_data(data, y_vars, x_vars, title):
    g = sns.PairGrid(
        data,
        y_vars=y_vars,
        x_vars=x_vars,
        height=4
    )
    g.map(sns.scatterplot, alpha=0.3 )
    g.fig.subplots_adjust(top=0.9)
    g.fig.suptitle(title)

### Метод corr_heat_map

Построение тепловой карты для значений параметров коррелиции

In [ ]:
def corr_heat_map(data, title):
    corr = data.corr()
    ax = sns.heatmap(corr, cmap = 'Wistia', annot= True)
    ax.title.set_text(title)
    display(corr['product'])


### Метод data_box_plot

Построение графиков диаграммы с усами для нескольких колонок

In [ ]:
def data_box_plot(data, columns):
    data_0[columns].plot(
        kind='box', 
        subplots=True, 
        sharey=False, 
        figsize=(10, 6)
    )
    # increase spacing between subplots
    plt.subplots_adjust(wspace=0.5) 
    plt.show()

### Метод cross_validation

Деление выборки на признаки и исследуемый параметр, кросс-валидация, выбор лучшей модели

In [ ]:
def cross_validation(data, cv):
    # first we separate data for features and target data
    features = data.drop(columns=['product','id'])
    target = data['product']
    # initilize best_mean_squarred_error to pick best model from all of them
    # picked NaN so on 1st comparance it picked it
    best_mean_squarred_error = np.NaN
    sample_size = int(len(data)/cv)
    # for comparing uses cross validation method
    for i in range(0, len(data), sample_size):
        # selecting valid data and train data
        valid_indexes = range(i, i + sample_size)
        features_valid = features.reindex(index = valid_indexes)
        target_valid = target.reindex(index = valid_indexes)
        features_train =  features.drop(index = valid_indexes)
        target_train = target.drop(index = valid_indexes)
        model = Pipeline(steps=[
            ("scaler", StandardScaler()),
            ("pca", PCA()),
            ("linear", LinearRegression())
        ])
        param_grid = {"pca__n_components": range(1,4)}
        search = GridSearchCV(
            model,
            param_grid = param_grid,
            scoring='neg_mean_squared_error'
        ).fit(features_train, target_train)
        
        if not(search.best_score_ <= best_mean_squarred_error):
            best_mean_squarred_error = search.best_score_
            best_model = search.best_estimator_
            best_valid_index = valid_indexes
    return ((-best_mean_squarred_error)**0.5,
            best_model,
            best_valid_index,
            best_model.predict(features.reindex(index = valid_indexes)),
            features,
            target,
            target_valid.reindex(index = valid_indexes)
           )

### Метод dummy_check

Сравнение с простым DummyRegressor, который выбирает средние значение

In [ ]:
def dummy_check(feature, feature_valid,  target, target_valid, model_rmse):
    dummy_regr = DummyRegressor(strategy="mean")
    dummy_regr.fit(feature, target)
    dummy_rmse = mean_squared_error(target_valid, dummy_regr.predict(target_valid)) ** 0.5
    print(f'Dummy predicted rmse - {dummy_rmse}, model predicted rmse - {model_rmse}')
    if dummy_rmse > model_rmse:
        print('Model works fine, still better than dummy')
    else:
        print('Model is not very helpful')

### Функция revenue

Функция подсчётов доходов

In [ ]:
def revenue(predicted_valid, n_developed_places, cash_per_product, budget):
    return predicted_valid.sort_values(ascending=False)[:n_developed_places].sum() * cash_per_product - budget

### Метод find_profit

Нахождение прибыли по предсказанному объёму нефти

In [ ]:
def find_profit(predicted_valid, n_researched_places,  n_developed_places, cash_per_product, budget ):
    profit = []
    state = np.random.RandomState(random_state)
    for i in range(n_bootstraps):
        researched_places = pd.Series(predicted_valid).sample(n=n_researched_places, replace=True, random_state=state)
        profit.append(revenue(researched_places, n_developed_places, cash_per_product, budget))
    return pd.Series(profit)
    

### Метод find_risk

In [ ]:
def find_risk(
    predicted_valid,
    n_researched_places,
    n_developed_places,
    cash_per_product,
    budget,
    title,
    damages_chances,
    confidence,
    n_bootstraps  
):
    profit = find_profit(predicted_valid, n_researched_places,  n_developed_places, cash_per_product, budget )
    fig = px.histogram(profit, title=title)
    fig.show()
    mean_profit = profit.mean()
    print(f'Mean profit - {mean_profit:.2f}\n')
    confidence_interval = st.t.interval(confidence , len(profit)-1, profit.mean(), profit.sem())
    print(f'Confidence Interval 95% - {confidence_interval}\n')
    risk_chance = (profit < 0).sum() / profit.count()
    print(f'Chance of Loses - {risk_chance}\n')
    if risk_chance > damages_chances:
        print('This region doesn\'t worth invistigation since risks are too high')
    else: 
        print('This region is potentially worthy invistigation')
    return profit


### Метод compare_bootstrap

In [ ]:
def compare_bootstrap(profit_0, profit_2, n_bootstraps):
    difference_ununited = profit_0.mean() - profit_2.mean()
    state = np.random.RandomState(random_state)
    count = 0
    for i in range(n_bootstraps):
        united_samples = pd.concat([profit_0, profit_2]) 
        subsample = united_samples.sample(frac=1,replace=True, random_state=state)
        subsample_0 =  subsample[:len(profit_0)]
        subsample_2 = subsample[len(profit_2):]
        bootstrap_difference = subsample_0.mean() - subsample_2.mean()
        if bootstrap_difference >= difference_ununited:
            count += 1

    pvalue = 1. * count / n_bootstraps
    print('p-value =', pvalue)

    if pvalue < alpha:
        print("Reject Null Hypothesis, Most likely mean profit is more")
    else:
        print("Wasn't able to reject Null Hypothesis, Most likely mean  profit is the same")

###  Исравленные функции

<div class="alert alert-block alert-warning">
<b>Комментарий студента:</b> 
    
    Новые исправленные функции
</div>

In [ ]:
def revenue_fixed(predicted_valid, n_developed_places, cash_per_product, budget, target_valid):
    top_predicted_places = predicted_valid.sort_values(ascending=False)[:n_developed_places].index
    return target_valid[top_predicted_places].sum() * cash_per_product - budget

In [ ]:
def find_profit_fixed(predicted_valid, n_researched_places,  n_developed_places, cash_per_product, budget, target_valid ):
    profit = []
    state = np.random.RandomState(random_state)
    for i in range(n_bootstraps):
        researched_places = pd.Series(predicted_valid, index = target_valid.index).sample(n=n_researched_places, replace=True, random_state=state)
        # profit.append(revenue(researched_places, n_developed_places, cash_per_product, budget)) - original
        profit.append(revenue_fixed(researched_places, n_developed_places, cash_per_product, budget, target_valid)) # - fixed
    return pd.Series(profit)
    

In [ ]:
def find_risk_fixed(
    predicted_valid,
    n_researched_places,
    n_developed_places,
    cash_per_product,
    budget,
    title,
    damages_chances,
    confidence,
    n_bootstraps,
    target_valid
):
    #profit = find_profit(predicted_valid, n_researched_places,  n_developed_places, cash_per_product, budget ) - original
    profit = find_profit_fixed(predicted_valid, n_researched_places,  n_developed_places, cash_per_product, budget, target_valid ) # - fixed
    fig = px.histogram(profit, title=title)
    fig.show()
    mean_profit = profit.mean()
    print(f'Mean profit - {mean_profit:.2f}\n')
    # confidence_interval = st.t.interval(confidence , len(profit)-1, profit.mean(), profit.sem()) - original 
    quantile = (1 - confidence) / 2
    lower = profit.quantile(quantile) 
    upper = profit.quantile(1- quantile)
    print(f'Confidence Interval 95% - {lower} - {upper}\n')
    #print(f'Confidence Interval 95% - {confidence_interval}\n')
    #risk_chance = (profit < 0).sum() / profit.count() - original
    risk_chance = (profit < 0).mean() # fixed
    print(f'Chance of Loses - {risk_chance}\n')
    if risk_chance > damages_chances:
        print('This region doesn\'t worth invistigation since risks are too high')
    else: 
        print('This region is potentially worthy invistigation')
    return profit


## Загрузка и подготовка данных

### Открываем файлы

In [ ]:
try:
    data_0 = pd.read_csv('/datasets/geo_data_0.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_0.csv')
try:
    data_1 = pd.read_csv('/datasets/geo_data_1.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_1.csv')
try:
    data_2 = pd.read_csv('/datasets/geo_data_2.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_2.csv')

### Изучение данных

#### Общий анализ данных

Для начала рассмотрим как вообще представлены данные.

In [ ]:
data_0.sample(10 ,random_state=12345)

In [ ]:
data_1.sample(10 ,random_state=12345)

In [ ]:
data_2.sample(10 ,random_state=12345)

In [ ]:
data_0.info()

In [ ]:
data_1.info()

In [ ]:
data_2.info()

`data_0`, `data_1`, `data_2` - базы данных, хранящие информацию по трём регионам по добычи нефти.

**Признаки**
- `id` — уникальный идентификатор скважины, представлен в виде строкового типа данных;
- `f0`, `f1`, `f2` — три признака точек (неважно, что они означают, но сами признаки значимы), представлены в виде вещественного типа данных;

**Целевой признак**
- `product` — объём запасов в скважине (тыс. баррелей), предаставлен в виде вещественного типа данных.

`f0`, `f1`, `f2`, `product` - количественные типы данных

`id` - категориальный тип данных

Имена колонок уже приведены к классическому змеиному регистру, поэтому их преобразование не требуется.

Также данных отсутствуют пропуски.

In [ ]:
print(f'Первая гео-локация\n {data_0.shape[1]} колонок;\n {data_0.shape[0]} строк.')
print(f'\nВторая гео-локация\n {data_1.shape[1]} колонок;\n {data_1.shape[0]} строк.')
print(f'\nТретья гео-локация\n {data_2.shape[1]} колонок;\n {data_2.shape[0]} строк.')

#### Уникальный индетификатор

Проверим является ли уникальный индетификатор уникальным в пределах одной гео-локации.

In [ ]:
data_0['id'].value_counts().count() != data_0['id'].count()

In [ ]:
data_0['id'].value_counts()

In [ ]:
data_1['id'].value_counts().count() != data_1['id'].count()

In [ ]:
data_1['id'].value_counts()

In [ ]:
data_2['id'].value_counts().count() != data_2['id'].count()

In [ ]:
data_2['id'].value_counts()

Похоже, что уникальный идентификатор не такой уж и уникальный. Следующим проверим, являются ли эти объекты полными дубликатами

In [ ]:
data_0[data_0.duplicated()]

In [ ]:
data_1[data_1.duplicated()]

In [ ]:
data_2[data_2.duplicated()]

Похоже что нет, тогда рассмотрим их в ручную.

In [ ]:
data_0[data_0['id'].duplicated(False)].sort_values(by='id')

In [ ]:
data_1[data_1['id'].duplicated(False)].sort_values(by='id')

In [ ]:
data_2[data_2['id'].duplicated(False)].sort_values(by='id')

Что же похоже никакой корреляции между дубликатами нет  - "одна и та же" скважина может иметь целевой признак в первой записи всего лишь 6, а в другой уже 184. Поскольку нет возможности проверить какой же из объектов корректный, то лучше будет просто отбросить все дубликаты.

In [ ]:
data_0.drop( data_0[data_0['id'].duplicated(False)].index, inplace=True )
data_0.reset_index(drop=True, inplace=True)
data_1.drop( data_1[data_1['id'].duplicated(False)].index, inplace=True )
data_1.reset_index(drop=True, inplace=True)
data_2.drop( data_2[data_2['id'].duplicated(False)].index, inplace=True )
data_2.reset_index(drop=True, inplace=True)

In [ ]:
print(f'Первая гео-локация\n {data_0.shape[1]} колонок;\n {data_0.shape[0]} строк.')
print(f'\nВторая гео-локация\n {data_1.shape[1]} колонок;\n {data_1.shape[0]} строк.')
print(f'\nТретья гео-локация\n {data_2.shape[1]} колонок;\n {data_2.shape[0]} строк.')

С дубликатами в пределах одной и той же локации разобрались. Что же касается проверки ID в меж-регионнах, то это скорее всего не потребуется, поскольку базам данных необязательно гарантировать уникальность id за пределами её, и совпадение их может просто быть вызвано случайностью, а не ошибкой.

ID в дальнейшем больше не будет рассматриваться, поскольку он не потребуется для обучения модели.

#### Признаки точек

In [ ]:
data_box_plot(data_0, ['f0','f1','f2'])

In [ ]:
data_0.describe().T

In [ ]:
data_1.describe().T

In [ ]:
data_2.describe().T

In [ ]:
pd.concat([data_0, data_1, data_2]).describe().T.iloc[:,[3,7]]

Для начала стоит отметить, что признаки не отмасштабириованы : для f0 - минимальные и максимальные значения -31 и 29, когда как для f2 - это -12 и 16. Также средние значение во невсегда равно нулю.

Дальше посмотрим как признак f0 распределен в различных гео-локациях:

In [ ]:
check_destribution(
    data_0['f0'],
    data_1['f0'],
    data_2['f0'],
    ['f0 for location 1', 'f0 for location 2', 'f0 for location 3'],
    'Comparision of f0 for different locations'
)

Только 3-ая геолокация похожа на нормальное распределение
Локация два имеет два пика около -10 и 10, а также сравнительный с другими огромный разброс в значениях
Локация первая - самая узкая ( т.е имеет наименьший разброс ), но она имеет аж 4 пика

In [ ]:
check_destribution(
    data_0['f1'],
    data_1['f1'],
    data_2['f1'],
    ['f1 for location 1', 'f1 for location 2', 'f1 for location 3'],
    'Comparision of f1 for different locations'
)

Тут ситуация крайне похоже на предудущие распределение, только стоит отметить, что распределение f1-параметра для второй локации не имеет двух пиков, но при этом оно смещено оносительно 0.

In [ ]:
check_destribution(
    data_0['f2'],
    data_1['f2'],
    data_2['f2'],
    ['f2 for location 1', 'f2 for location 2', 'f2 for location 3'],
    'Comparision of f2 for different locations'
)

Распределние f2-параметра для локации 1-й и 3-й практически идентичны, оба имеет смещение относительно 0 около 3, похожи на нормальное распределение с относительно большим разбросом. В то время как распределение для второй локации крайне странное, надо бы рассмотреть его по подробнее

In [ ]:
ax = data_1.query('2.9 < f2 < 3.1')['f2'].hist(figsize=(15,5), bins=30)
ax.title.set_text('Destribution of f2 for 2nd location around 3')
plt.show()

In [ ]:
fig = px.histogram(data_1['f2'], title='Destribution of f2 for 2nd location')
fig.show()

Похоже, что общее распределение параметра представлено как совокупность 6 различных пуассоновских распределений, в различных точках. Поскольку нам ничего неизвестно из условия задачи о том, что из себя представляет различные геолокации или же параметры, то сложно оценить причину, но по сравнению с остальными локациями вторая, точно наиболее странная.

#### Объём нефти

In [ ]:
check_destribution(
    data_0['product'],
    data_1['product'],
    data_2['product'],
    ['product for location 1', 'product for location 2', 'product for location 3'],
    'Comparision of products for different locations'
)


In [ ]:
data_1['product'].value_counts()

Во первых хочется отметить, что локация два опять принимает крайне странные значения. То есть, чтобы случайная величина вроде объёма нефти принимала абсолютные одинаковые значения с точностью до миллионых и при этом распределение будет равномерным звучит крайне маловероятно.

Во вторых же распределение первой и третий локации крайне похоже, они имеют одинаковое количество пиков, а также они находится приблизительно в одинаковых местах

Дальше предварительно рассмотрим как этот параметр зависит от других

In [ ]:
corr_heat_map(data_0, 'Correlation Matrix for Location 1')

In [ ]:
corr_heat_map(data_1, 'Correlation Matrix for Location 2')

In [ ]:
corr_heat_map(data_2, 'Correlation Matrix for Location 3')

In [ ]:
pair_plot_data(data_0,['product'], ['f0', 'f1', 'f2'], 'product by different parametrs for location 1')

In [ ]:
pair_plot_data(data_1,['product'], ['f0', 'f1', 'f2'], 'product by different parametrs for location 2')

In [ ]:
pair_plot_data(data_2,['product'], ['f0', 'f1', 'f2'], 'product by different parametrs for location 3')

Вторая локация опять выделяется из других. Она имеет аномальный высокий коэффициент корреляции. По крайней мере с коэффициентом корреляции 0.999397 f2-параметра и объёма нефти, можно будет ожидать высокой точности, что свидетельсвует о **утечки целевого
признака**

Остальные локации тоже имеют относительно высокой коэффициент кореляции между f2-параметра и объёма нефти, но это по сравнению с другими параметрами, которые не превышают 0.2

Влияние f0, f1 параметров для третий локации крайне мизерное. Но можно заметить, что если они больше 2.5 у.е то они имеют выше вероятность не получить совсем низкое значения объёма нефти.

#### Вывод по изучению данных

- Избавились от смысловых дубликатов в базах данных;

- Была определена ненормализованность данных, которые надо быдет приобразовать на этапе обработке;

- Обнаружен потенциальный самый значимый параметр модели - f2;

- Данные по объёму для всех локации приблизитильно сбалансированы;

- Была обнаружена аномальность данных для локации номер 2 - случайная величина вроде объёма нефти принимает одно из 12 фиксированных значений, также неестественно высокий коэффициент корреляции между ним и параметром f2, *основываясь на этом можно было бы отбросить эти данные как некорректные*, но тут надо понимать, что отбрасывать треть от всех данных это крайне серьёзное решение, которое условный `Junior Data Scientist` вряд ли может единолично принять без обсуждения этого с Data Analytics, поэтому на **данный момент будет справедливей оставить эти данные**, но при этом **в заключительном выводе отметить про аномальность этих данных**

<div class="alert alert-block alert-success">
<b>Успех:</b> Данные загружены и осмотрены корректно. Хорошо, что дубликаты по id были удалены! Радует, что распределения и корреляции были визуализированы!
</div>

## Обучение и проверка модели

### Обучение модели

> Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).

Из условия задачи, будем пользоваться для обучения только линейной регрессией

> Разбейте данные на обучающую и валидационную выборки в соотношении 75:25.

Также воспользуемся кросс-валидационной техникой, чтобы обучение было более точным

In [ ]:
rmse = {}
rmse['geo_location_1'], model_0, valid_index_0, predict_valid_0, features_0, target_0, target_valid_0 = cross_validation(data_0, 4)
rmse['geo_location_2'], model_1, valid_index_1, predict_valid_1, features_1, target_1, target_valid_1 = cross_validation(data_1, 4)
rmse['geo_location_3'], model_2, valid_index_2, predict_valid_2, features_2, target_2, target_valid_2 = cross_validation(data_2, 4)

>  Напечатайте на экране средний запас предсказанного сырья и RMSE модели.

In [ ]:
pd.DataFrame.from_dict(rmse, orient='index', columns=['RMSE'])

Локация два не перестает поражать, всего лишь 0.8 - RMSE. 

Что касается остальных локации, то первая показала себя немного лучше, но это можно связать, с тем, что изначально средние максимальные значения были больше в оригинальной базе данных.

In [ ]:
mean_data = {
    'geo_location_1': predict_valid_0.mean(),
    'geo_location_2': predict_valid_1.mean(),
    'geo_location_3': predict_valid_2.mean()
}
pd.DataFrame.from_dict(mean_data, orient='index', columns=['predicted mean'])

Хоть и точность предсказаний у второй локации гораздо выше, но при этом средние значения предсказанных значительно меньше

### Проверка на адекватность

Значение  37 или же 40 для RMSE - это много или же мало, сложно сказать без каких либо сравнений с другими данными. Для этого проверим на адекватность сравнив их с простыми моделями предсказания.

In [ ]:
dummy_check(
    features_0.drop(valid_index_0),
    features_0.loc[valid_index_0],
    target_0.drop(valid_index_0),
    target_valid_0,
    rmse['geo_location_1']
)

In [ ]:
dummy_check(
    features_2.drop(valid_index_2),
    features_2.loc[valid_index_2],
    target_2.drop(valid_index_2),
    target_valid_2,
    rmse['geo_location_3']
)

Что же похоже модели по крайней мере работают лучше, чем Dummies. 

### Заключение по обучению и проверки модели

Для обучения была выбрана модель линейной регрессии, чтобы обучение оказалось более точным использовался метод кросс-валидации, разделяя выборки на четыре равный по размеру выборки, каждую из которых последовательно выбирали в качестве валидационной для нахождения более точной модели.  

Для проверки адекватность моделей, их сравнивали с простой моделью, которая выдавала средний результат, по обучающий выборки. В итоге наши обученные модели оказались видимо лучше, чем такие. 

Метрика ошибки для первой локации оказалось немного лучше, чем у третий, но в то же время предсказанное среднее было больше у третий.

В тоже время вторая локация показала практически безупречную точность. Но её предсказанное среднее оказалось значительно меньше.

<div class="alert alert-block alert-success">
<b>Успех:</b> Модели обучены корректно. Радует, что ты активно используешь функции. Это помогает избежать дублирования кода, а значит уменьшает вероятность совершить ошибку/опечатку!
</div>

## Подготовка к расчёту прибыли

### Условия задачи

> При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.

> Бюджет на разработку скважин в регионе — 10 млрд рублей.

> При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.

> После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

### Подготовка

> Все ключевые значения для расчётов сохраните в отдельных переменных.

In [ ]:
budget = 10_000_000_000
n_researched_places = 500
n_developed_places = 200
cash_per_product = 450_000
damages_chances = 0.025
n_bootstraps = 1000
confidence = 0.95


<div class="alert alert-block alert-info">

<b>Совет: </b> В названиях константных переменных лучше использовать только БОЛЬШИЕ буквы. Это соглашение между программистами: https://pythonworld.ru/osnovy/pep-8-rukovodstvo-po-napisaniyu-koda-na-python.html#id29 .
</div>

In [ ]:
BUDGET = 10_000_000_000
N_RESEARCHED_PLACES = 500
N_DEVELOPED_PLACES = 200
CASH_PER_PRODUCT = 450_000
DAMAGES_CHANCES = 0.025
N_BOOTSTRAPS = 1000
CONFIDENCE = 0.95


<img src="https://images.emojiterra.com/google/android-pie/512px/1f5d2.png" align=left width=44, heigth=44>
<div class="alert alert-block alert-warning">
<b> Комментарий студента </b>
    
Добавил новые переменные
</div>

> Рассчитайте достаточный объём сырья для безубыточной разработки новой скважины. Сравните полученный объём сырья со средним запасом в каждом регионе

$$
  Sufficied Product =  \frac{Budget}{Number Of Developed Places * Cash Per Product} \\
$$

In [ ]:
sufficied_product = BUDGET / N_DEVELOPED_PLACES / CASH_PER_PRODUCT
print(f'Sufficied product to not lose money per place - {sufficied_product:.2f}')
print('\nAverage product in location 1 - {:.2f}'.format(data_0['product'].mean()))
print('\nAverage product in location 2 - {:.2f}'.format(data_1['product'].mean()))
print('\nAverage product in location 3 - {:.2f}'.format(data_2['product'].mean()))

**Вывод:**
Как можно заметить, ни одна из локаций не имеет в среднем достаточно ресурсов, чтобы не потерять деньги. Поэтому нельзя просто выбирать локации наугад, необходимо для этого воспользоваться нашими обученными моделями, чтобы с наименьшей вероятностью понести убытки.

<div class="alert alert-block alert-success">
<b>Успех:</b> Точка безубыточности найдена верно, сравнение проведено!
</div>

## Расчёт прибыли и рисков 

### Устранение регионов с слишков высоким риском убытков

**Ошибочный код**
```python
profit_0 = find_risk(
    predict_valid_0,
    n_researched_places,
    n_developed_places,
    cash_per_product, 
    budget,
    'Histogram for Profit of Location 1',
    damages_chances,
    confidence,
    n_bootstraps,
    
)
```

In [ ]:
profit_0 = find_risk_fixed(
    predict_valid_0,
    n_researched_places,
    n_developed_places,
    cash_per_product, 
    budget,
    'Histogram for Profit of Location 1',
    damages_chances,
    confidence,
    n_bootstraps,
    target_valid_0
)

<div class="alert alert-block alert-danger">
<b>Ошибка:</b> Для выбора 200 лучших скважин нужно использовать предсказанные значения. А вот для подсчета прибыли должны использоваться настоящие значения соответствующих ячеек.
</div>


<div class="alert alert-block alert-danger">
<b>Ошибка:</b> Для подсчета доверительного интервала стоит использовать метод квантилей, так как мы реализуем технику бутстрап.
</div>

<div class="alert alert-block alert-info">

<b>Совет: </b> Риск можно было найти чуть проще: `(values < 0).mean()`, если объект типа pandas Series или numpy array.
</div>

<img src="https://images.emojiterra.com/google/android-pie/512px/1f5d2.png" align=left width=44, heigth=44>
<div class="alert alert-block alert-warning">
<b> Комментарий студента </b>
    
Исправил
</div>

In [ ]:
profit_1 = find_risk_fixed(
    predict_valid_1,
    n_researched_places,
    n_developed_places,
    cash_per_product, 
    budget,
    'Histogram for Profit of Location 2',
    damages_chances,
    confidence,
    n_bootstraps,
    target_valid_1
)

In [ ]:
profit_2 = find_risk_fixed(
    predict_valid_2,
    n_researched_places,
    n_developed_places,
    cash_per_product, 
    budget,
    'Histogram for Profit of Location 3',
    damages_chances,
    confidence,
    n_bootstraps,
    target_valid_2
)

Похоже, что только локации 1 и 3 отпадают в качестве исследуемых локации, из-за слишком высоких рисков.

### Выбор локации основываясь на прибыли


Поскольку 2 локация показала подозрительные значения для объёма нефти, хотелось бы проверить остальные локации, какая из них наиболее прибыльная на всякий случай, риск потерь у них приблизительно одиковый

- Нулевая гипотеза: прибыль в локации 1 и в 3 не отличается;

- Альтернативная гипотеза: прибыль в локации 1 выше;

In [ ]:
compare_bootstrap(profit_0, profit_2, n_bootstraps)

В итоге разница в прибыли у них наиболее вероятней всего была вызвана случайностью

## Заключение

По итогу исследования и обучения модели локация номер 2 была определена как потенциально наиболее прибыльная локация (467 млн рублей) с приемлимым показателем рисков 0.9%. Но важно отметить, что эта локации имеет крайне неестественное распределение, поскольку вместо случайного распределения объёма нефти, он принимает одно из 12 фиксированных значений. Также имеет аномальный высокий параметр корреляции для одного из признаков, крайне близкого к 1. Низкие риски у этой локации как раз и объясняются, тем что из-за утечки целевого признака, модель смогла предсказать значение объёма у исследуемых точек с филигранной точностью и тем самым случайно не выбрать, те места где мало ресурсов,

Поэтому слудует рассмотреть потенциальное принятие ко вниманию и другие локации, хотя они и имеют риски превосходящие необходимый минимум в 2.5% - 6.5%.

<img src="https://em-content.zobj.net/thumbs/120/toss-face/342/person-bowing_1f647.png" align=left width=44, heigth=44>
<div class="alert alert-block alert-warning">
<b> Заключительная благодарность. </b>
    
Спасибо ещё раз за комментарии, благодаря Вам смог осознать, что расчёт прибыли был совершён категорически не верно.
</div>

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Выполнен шаг 1: данные подготовлены
- [ ]  Выполнен шаг 2: модели обучены и проверены
    - [ ]  Данные корректно разбиты на обучающую и валидационную выборки
    - [ ]  Модели обучены, предсказания сделаны
    - [ ]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [ ]  На экране напечатаны результаты
    - [ ]  Сделаны выводы
- [ ]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [ ]  Для всех ключевых значений созданы константы Python
    - [ ]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [ ]  По предыдущему пункту сделаны выводы
    - [ ]  Написана функция расчёта прибыли
- [ ]  Выполнен шаг 4: посчитаны риски и прибыль
    - [ ]  Проведена процедура *Bootstrap*
    - [ ]  Все параметры бутстрепа соответствуют условию
    - [ ]  Найдены все нужные величины
    - [ ]  Предложен регион для разработки месторождения
    - [ ]  Выбор региона обоснован